In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
from great_barrier_reef import StarfishDatasetAdaptor

ModuleNotFoundError: No module named 'great_barrier_reef'

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import patches

def get_rectangle_edges_from_pascal_bbox(bbox):
    xmin_top_left, ymin_top_left, xmax_bottom_right, ymax_bottom_right = bbox

    bottom_left = (xmin_top_left, ymax_bottom_right)
    width = xmax_bottom_right - xmin_top_left
    height = ymin_top_left - ymax_bottom_right

    return bottom_left, width, height

def draw_pascal_voc_bboxes(
    plot_ax,
    bboxes,
    get_rectangle_corners_fn=get_rectangle_edges_from_pascal_bbox,
):
    for bbox in bboxes:
        bottom_left, width, height = get_rectangle_corners_fn(bbox)

        rect_1 = patches.Rectangle(
            bottom_left,
            width,
            height,
            linewidth=4,
            edgecolor="black",
            fill=False,
        )
        rect_2 = patches.Rectangle(
            bottom_left,
            width,
            height,
            linewidth=2,
            edgecolor="white",
            fill=False,
        )

        # Add the patch to the Axes
        plot_ax.add_patch(rect_1)
        plot_ax.add_patch(rect_2)

def show_image(
    image, bboxes=None, draw_bboxes_fn=draw_pascal_voc_bboxes, figsize=(10, 10)
):
    fig, ax = plt.subplots(1, figsize=figsize)
    ax.imshow(image)

    if bboxes is not None:
        draw_bboxes_fn(ax, bboxes)

    plt.show()

In [ ]:
from pathlib import Path

dataset_path = Path('../data/train_images/')
list(dataset_path.iterdir())

In [ ]:
data_df = pd.read_csv('../data/train.csv')

In [ ]:
data_df['annotations'] = data_df['annotations'].apply(literal_eval)

In [ ]:
def convert_string_annotation_to_json(annotation: str):
    pass

In [ ]:
from ast import literal_eval

In [ ]:
literal_eval(data_df.loc[data_df['annotations']!='[]', 'annotations'].values[0])

In [ ]:
data_df['annotations'][0]

In [ ]:
import PIL

In [ ]:
image_name = 'video_'+str(data_df['video_id'].values[0])+'/'+str(data_df['video_frame'].values[0])+'.jpg'
images_dir_path = Path('../data/train_images/')

In [ ]:
type(data_df.apply(lambda x: 'video_{}/{}.jpg'.format(x['video_id'], x['video_frame']), axis=1).values)

In [ ]:
image = PIL.Image.open(images_dir_path / image_name)

In [ ]:
from torch.utils.data import Dataset



    
    
class CorallDatasetAdaptor(Dataset):
    def __init__(
        self, 
        annotations_dataframe: pd.DataFrame,
        images_dir_path: str = '../data/train_images/', 
            ):
        self.images_dir_path = Path(images_dir_path)
        self.annotations_df = annotations_dataframe
        self.image_paths = self.prepare_image_paths()
        
    def prepare_image_paths(self):
        image_paths = self.annotations_df.apply(
            lambda x: 'video_{}/{}.jpg'.format(x['video_id'], x['video_frame']), 
            axis=1
        )
        return image_paths
        
    def __len__(self) -> int:
        return len(self.image_paths)
    
    def get_image_and_labels_by_idx(self, index):
        image_name = self.image_paths[index]
        image = PIL.Image.open(self.images_dir_path / image_name)

        return image, image_name
    
    def show_image(self, index):
        image, image_name = self.get_image_and_labels_by_idx(index)
        print(f"image_name: {image_name}")
        self._show_image(image)
        
    def _show_image(self, image, figsize=(10, 10)):
        fig, ax = plt.subplots(1, figsize=figsize)
        ax.imshow(image)

        plt.show()

In [ ]:
dataset = CorallDatasetAdaptor(data_df)

In [ ]:
dataset.show_image(0)

In [ ]:
len(dataset)